# Operator splitting 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-whitegrid')

import matplotlib as mpl
mpl.rc('lines', linewidth=2)

from nm_lib import nm_lib as nm

## 1- OS precision

Solve the following Burgers' equation: 

$$\frac{\partial u}{\partial t} = - a \frac{\partial u}{\partial x} - b \frac{\partial u}{\partial x}   \tag{1}$$

following exersize [2b](https://github.com/AST-Course/AST5110/blob/main/ex_2b.ipynb). where $x[x_0, x_f]$ with $x_0 = −2.6$, $x_f = 2.6$, $a=-0.7$ and $b=-0.3$, periodic boundary conditions and with initial condition:

$$u(x,t=t_0) = \cos^2 \left(\frac{6 \pi x}{5} \right) / \cosh(5x^2)  \tag{2}$$

Solve the evolution for the following four different approaches: 

- 1 With additive OS.  

- 2 With Lie-Trotter OS. 

- 3 With Strang OS.

- 4 Without an operator splitting and single time-step method but add the to terms: 

$$\frac{\partial u}{\partial t} = - (a+b) \frac{\partial u}{\partial x}$$

for $nump=256$ and 100 steps.

_Suggestion_: use the Lax-method scheme for all cases with `deriv_cent`. Make sure the boundaries are properly selected.

Fill in the function `osp_LL_Add`, `osp_LL_Lie`, and `osp_LL_Strang`.

Start with $cfl\_cut = 0.4$ and increase up to $0.9$.  

Which OS schemes are stable? Which one is more diffusive? Why?

In [ ]:
def u_0(x: np.ndarray, t: float = 0) -> np.ndarray:
    r"""
    Initial condition for the advection equation.

    Parameters
    ----------
    x : `array`
        the x-axis.
    t : `float`
        the time.
    
    Returns
    -------
    `array`
        the initial condition.
    """
    return np.cos(6*np.pi*x / 5)**2 / np.cosh(5*x**2)

In [ ]:
x0 = -2.6
xf = 2.6
a = -0.7
b = -0.3

nump = 256 # number of grid points

# xx = np.linspace(x0, xf, nump)

xx = np.arange(nump) / (nump - 1.0) * (xf - x0) + x0

In [ ]:
# Setup for animation
from matplotlib import animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

def init(): 
    axes.plot(xx,unnt_Add[:,0], label='Add')
    axes.plot(xx,unnt_Lie[:,0], label='Lie')
    axes.plot(xx,unnt_LL_Strang[:,0], label='LL_Strang')
    axes.plot(xx,unnt_ab[:,0], label='Lax (a+b)')
    axes.legend(loc='upper right', frameon=True, framealpha=0.5)

def animate(i):
    axes.clear()
    axes.plot(xx,unnt_Add[:,i], label='Add')
    axes.plot(xx,unnt_Lie[:,i], label='Lie')
    axes.plot(xx,unnt_LL_Strang[:,i], label='LL_Strang')
    axes.plot(xx,unnt_ab[:,i], label='Lax (a+b)')
    axes.set_title('t=%.2f'%t[i])
    axes.legend(loc='upper right', frameon=True, framealpha=0.5)
    axes.set_ylim(-0.05, 1.05)

In [ ]:
nt = 100
cfl_cut = 0.4

t, unnt_Add = nm.ops_Lax_LL_Add(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_Lie = nm.ops_Lax_LL_Lie(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_LL_Strang = nm.ops_Lax_LL_Strang(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_ab = nm.evolv_Lax_adv_burgers(xx, u_0(xx), nt=nt, a=a+b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))
anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
plt.close()
HTML(anim.to_jshtml())

<span style="color:#78AE7E">

Above is a comparison of the four different methods over time, when `cfl_cut=0.4`. We see that the Add method is the first to blow up, beginning at around $t=0.25$. The Lie method also blows up eventually, but this doesn't start until about $t=0.76$. Both the Lie and Add methods seem to diffuse at an equal pace before the Add method blows up, though the Lie method diffuses a little bit faster. The Lie and Add methods are both 1st-order methods, which can explain why they are both unstable.

Both the Strang method and the Lax method with $a+b$ added do not blow up during the course of the simulation, but we see that the Strang method diffuses much quicker than the Lax method. The Strang method and the Lax method are both 2nd-order methods, and thus it makes sense that both of them are stable.

The four solutions move across the simulated span at different paces, due to the difference in the coefficients $a$ and $b$. The Lie method moves the fastest, followed by the Add method, then the Strang method, and finally the Lax method.

<span>

In [ ]:
nt = 100
cfl_cut = 0.9

t, unnt_Add = nm.ops_Lax_LL_Add(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_Lie = nm.ops_Lax_LL_Lie(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_LL_Strang = nm.ops_Lax_LL_Strang(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_ab = nm.evolv_Lax_adv_burgers(xx, u_0(xx), nt=nt, a=a+b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))
anim = FuncAnimation(fig, animate, interval=50, frames=nt, init_func=init)
plt.close()
HTML(anim.to_jshtml())

<span style="color:#78AE7E">

In the case where `cfl_cut=0.9`, we see that the amplitudes of the Lie and Add methods begin to increase immediately, showing that they are unstable solutions in this case. The Lie method blows up immediately, while the Add method begins to blow up around $t=0.55$. 

As with `cfl_cut=0.4`, the Strang and Lax methods are stable solutions, and they behave in the same way. 

<span>

## 2- When does it not work? 

Use OS-Strang from the previous exercise and try to apply a predictor-corrector explicit method. 
To facilitate this exercise, `nm_lib` already includes the predictor-corrector Hyman method, which is included Bifrost (`Hyman`). Fill in the function `osp_Lax_LH_Strang`. Use the same setup as the previous exercise but with $nump=512$, $500$ steps, and $cfl\_cut=0.8$. 

What do you notice? 

__Optional__: Apply the Hyman predictor-corrector explicit method to the Burgers equation and check if the following is true: 

$$u^{n+1} = F\, u^{n}\Delta t \approx G\, u^{n}\Delta t+H\, u^{n}\Delta t$$

In [ ]:
x0 = -2.6
xf = 2.6
a = -0.7
b = -0.3

nump = 512 # number of grid points
nt = 500 
# nt = 10
cfl_cut = 0.8

xx = np.linspace(x0, xf, nump)

t, unnt = nm.ops_Lax_LH_Strang(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])
t, unnt_LL = nm.ops_Lax_LL_Strang(xx, u_0(xx), nt=nt, a=a, b=b, cfl_cut=cfl_cut, ddx=nm.deriv_cent, bnd_limits=[1, 1])

In [ ]:
def init(): 
    axes.plot(xx,unnt[:,0], label='LH')
    axes.plot(xx,unnt_LL[:,0], label='LL')
    axes.legend(loc='upper right', frameon=True, framealpha=0.5)

def animate(i):
    axes.clear()
    axes.plot(unnt[:, 320][int(len(unnt[0])/2)], 'r+')
    # axes.plot(0, , 'ro')
    # axes.axhline(unnt[:, 50], -2.6, 2.6, c='k', ls='--')
    axes.plot(xx,unnt[:,i], label='LH')
    axes.plot(xx,unnt_LL[:,i], label='LL')
    axes.set_title('t=%.2f'%t[i])
    axes.set_ylim(-0.05, 1.05)
    axes.legend(loc='upper right', frameon=True, framealpha=0.5)

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(7, 3))
anim = FuncAnimation(fig, animate, interval=30, frames=nt, init_func=init)
plt.close()
HTML(anim.to_jshtml())

<span style="color:#78AE7E">

Here we have compared the Strang method with (LH) and without (LL) the Hyman predictor-corrector method. Both solutions are stable, as is expected. 

We can see, however, that the Hyman method seems to be moving ahead of the LL-method, but at around the same pace. By marking one spot we can see that both solutions have diffused almost equally much at this point on the x-axis, though if you look very carefully, it seems the LL method has just a *little* higher amplitude at that point. Below we take an even closer look at this.
<span>

In [ ]:
n = 321

plt.plot(xx, unnt[:, n], label=f'LH (t={t[n]:.2f})')
plt.plot(xx, unnt_LL[:, n+105], label=f'LL (t={t[n+105]:.2f})')
plt.xlim(-1, 1); plt.ylim(0.51, 0.56)
plt.legend(frameon=True, framealpha=1)

<span style="color:#78AE7E">

This figure shows a very zoomed in view of the two solutions at the respective time steps which they both pass the same point on the x-axis, after moving one "round" in the spatial simulation scope, and ending up back at $x=0$. 

Here we see that the LL-method indeed has a slightly higher amplitude than the LH-method. This leads me to believe that the LL-method without the Hyman predictor-corrector is a little bit less diffusive in this case. 

I am not sure why this is the case, as I struggled a bit to understamd the Hyman method. It could very well be that I implemented it wrong too! 

<span>